In [7]:
import os
import random


class Grammar:
    
    non_terminal_symbols = ['S', 'B', 'C', 'D'] 
    terminal_symbols = ['a', 'b', 'c']
    set_of_productions = tuple([
        ('S', 'aB'),
        ('B', 'bS'),
        ('B', 'aC'),
        ('B', 'c'),
        ('C', 'bD'),
        ('D', 'c'),
        ('D', 'aC')
    ])

    def getGeneratedString(self,n,s):
        result = ""
        if(n == 1):
            if(s == "S"):
                result = "ac"
            elif(s == "B"):
                result = "c"
            elif(s=="C"):
                result = "bc"
            elif(s=="D"):
                result = "c"
            return result
        else:
            if(s=="C"):
                result = 'b'+self.getGeneratedString(n-1,"D")
            elif(s == "S"):
                result = 'a'+self.getGeneratedString(n,"B")
            elif(s == "B"):
                choice = random.randint(0,1);
                if(choice == 0):
                    result = 'b'+self.getGeneratedString(n-1,"S")
                elif(choice == 1):
                    result = 'a'+self.getGeneratedString(n-1,"C")
            elif(s=="D"):
                result = 'a'+self.getGeneratedString(n-1,"C")

            return result
    def toFiniteAutomaton(self):
        g_dict = {}
        count = 0

        for element in self.non_terminal_symbols:
            g_dict[element] = "q{}".format(count)
            count += 1

        g_dict['Q'] = "q{}".format(count + 1)



        __set_of_productions = {}

        for nonterminal in self.set_of_productions:
            __set_of_productions[nonterminal[0]] = [production[1]
                                                    for production in self.set_of_productions if production[0] == nonterminal[0]]

        finite_automaton = dict()
        for non_terminal in __set_of_productions:
            finite_automaton[g_dict[non_terminal]] = []
            for transition in __set_of_productions[non_terminal]:
                if len(transition) == 2:
                    finite_automaton[g_dict[non_terminal]].append(
                        (transition[0], g_dict[transition[1]]))
                else:
                    finite_automaton[g_dict[non_terminal]].append((transition, 'Q'))
        return finite_automaton
    
    
class FiniteAutomaton:
    adjacency_matrix = dict()
    start_node = 'q0'
    def setAdjacencyMatrix(self,finite_automaton):
        self.adjacency_matrix = finite_automaton
        
    def stringBelongToLanguage(self,string):
        
        # Set the current node to the start one.
        current_node = self.start_node
        for c in string:
            if current_node == 'Q':
                return False

            for weight, adj_node in self.adjacency_matrix[current_node]:
                if c == weight:
                    current_node = adj_node
                    break
            else:
                return False

        # Check if the last node is other then Empty.
        if current_node != 'Q':
            for prod in self.adjacency_matrix[current_node]:
                # If there exists a prod with the exact weight like the last character
                # and the adjacency node empty.
                if prod[0] == string[-1] and prod[1] == 'Q':
                    return True

        # Return True or False if the current (last) node is the empty one.
        return current_node == 'Q'

    
    # DEBUG:
    
gram = Grammar()
ff = gram.toFiniteAutomaton()
f_a = FiniteAutomaton()
f_a.setAdjacencyMatrix(ff)

print('Finite Automaton:')
print(ff)
for i  in range(5):
    gen_string = gram.getGeneratedString(i+1,"S")
    print(gen_string)
    print(f_a.stringBelongToLanguage(gen_string))
    
print(f_a.stringBelongToLanguage("acc"))

Finite Automaton:
{'q0': [('a', 'q1')], 'q1': [('b', 'q0'), ('a', 'q2'), ('c', 'Q')], 'q2': [('b', 'q3')], 'q3': [('c', 'Q'), ('a', 'q2')]}
ac
True
abac
True
ababac
True
aababc
True
abaababc
True
False
